In [1]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split

from VQLS import QuantumSVM

In [2]:
auxiliaryQubit = 0
shots = 100000 # 100000
gamma = 0.01 # regularization parameter
testSize = 0.2

In [3]:
svm = QuantumSVM(auxiliaryQubit=auxiliaryQubit, shots=shots)

In [ ]:
X, y = datasets.make_classification(n_samples=50, n_features=2, n_informative=2, n_redundant=0, random_state=45)
y = np.where(y == 0, -1, 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testSize, random_state=42)
subsetSize = 7
subsetIndices = np.random.choice(np.arange(X_train.shape[0]), subsetSize, replace=False)
xSubset = np.array([ X_train[i] for i in subsetIndices ])
ySubset = np.array([ y_train[i] for i in subsetIndices ])

lssvm = ClassicalLSSVM(xSubset,ySubset, gamma, "linearKernel")
lssvm_matrix = lssvm.combineTopBottomMatrices(xSubset)
yVector = lssvm.prepareLabels(ySubset)

print ("LS-SVM Matrix:", lssvm_matrix)

pauliOp = SparsePauliOp.from_operator(lssvm_matrix)
paulis = pauliOp.paulis
print(paulis)

coefficientSet = svm.getMatrixCoeffitients(pauliOp)
print (coefficientSet)

out_f = svm.minimization(paulis=paulis, coefficientSet = coefficientSet, totalNeededQubits = pauliOp.num_qubits+2, bVector = yVector, quantumSimulation=True)
circ = QuantumCircuit(pauliOp.num_qubits,pauliOp.num_qubits)
o = svm.ansatzTest(circ, out_f)

svm.getApproximationValue(lssvm_matrix, yVector, o)
svm.plotCost()

In [ ]:
print (o)
print (lssvm_matrix.dot(o))

svm.getApproximationValue(lssvm_matrix, yVector, o)
svm.plotCost()

In [ ]:
subsetIndicesTest = np.random.choice(np.arange(X_test.shape[0]), subsetSize, replace=False)
xTestSubset = []
yTestSubset = []
for i in subsetIndicesTest:
    xTestSubset.append(X_test[i])
    yTestSubset.append(y_test[i])

xTestSubset = np.array(xTestSubset)
yTestSubset = np.array(yTestSubset)
print(o)
b = o[0]
print(b)
w = []
for i in range (len(o)):
    if (i != 0):
        w.append(o[i])
print (w)
predictions = lssvm.predict(xSubset, xTestSubset, w, b)
print("Predictions:", predictions)
print("Accuracy:", lssvm.accuracy(yTestSubset, predictions))